In [1]:
import numpy as np
import os

In [2]:
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

In [3]:
bst_shap=[]
clf_shap=[]
logreg_shap=[]
rf_shap=[]
for i in findAllFile("/home/zyxing/adult_income/shaps"):
    name=os.path.basename(i)
    # print(i)
    if name.startswith("bst"):
        bst_shap.append(np.load(i))
    elif name.startswith("clf"):
        clf_shap.append(np.load(i))
    elif name.startswith("logreg"):
        logreg_shap.append(np.load(i))
    elif name.startswith("rf"):
        rf_shap.append(np.load(i))
bst_shap,clf_shap,logreg_shap,rf_shap=np.array(bst_shap),np.array(clf_shap),np.array(logreg_shap),np.array(rf_shap)
print(bst_shap.shape,clf_shap.shape,logreg_shap.shape,rf_shap.shape)


(20, 22792, 14) (94, 22792, 14) (20, 22792, 14) (20, 22792, 14)


In [4]:
all_shap_data=np.concatenate([bst_shap,clf_shap,rf_shap],axis=0)
all_shap=np.mean(all_shap_data,axis=0)
print(all_shap.shape)

(22792, 14)


In [5]:
rand_ints=np.random.choice(20,5,replace=False)
all_input_shap=logreg_shap[rand_ints,:,:]
print(rand_ints)
all_input_shap=all_input_shap.transpose([1,0,2])
print(all_input_shap.shape) 

[19 11  8  5 15]
(22792, 5, 14)


In [6]:
from torch.utils.data import Dataset,DataLoader
import torch

class cls_dataset(Dataset):
    def __init__(self,x_,y_):
        self.X=torch.tensor(x_).to(torch.float)
        self.y=torch.tensor(y_).to(torch.float)
    def __getitem__(self, index):
        return self.X[index],self.y[index],index
    def __len__(self):
        return len(self.X)

In [7]:
#500 22792
#1000 14
from sklearn import decomposition
from sklearn.model_selection import train_test_split
import joblib
pca=decomposition.PCA(n_components=0.99)
partial_shaps_pca=pca.fit_transform(all_input_shap.reshape(22792,-1))
joblib.dump(pca,"partial_shaps_pca.m")
pca=decomposition.PCA(n_components=0.99)
shaps_mean_pca=pca.fit_transform(all_shap.reshape(-1,14))
joblib.dump(pca,"shaps_mean_pca.m")
input_dim=partial_shaps_pca.shape[1]
output_dim=shaps_mean_pca.shape[1]

print(partial_shaps_pca.shape,shaps_mean_pca.shape)
X_train, X_test, y_train, y_test = train_test_split(partial_shaps_pca, shaps_mean_pca, test_size = 0.3, shuffle=False)
print(X_test.shape)
train_datasets=cls_dataset(X_train,y_train)
test_datasets=cls_dataset(X_test,y_test)
train_dataloader = DataLoader(train_datasets,batch_size=128,shuffle=True)
test_dataloader = DataLoader(test_datasets,batch_size=128,shuffle=True)

(22792, 7) (22792, 9)
(6838, 7)


In [8]:
import torch.nn as nn
class pca_Net(nn.Module):
    def __init__(self):
        super(pca_Net,self).__init__()
        self.fc1=nn.Linear(input_dim,11)
        self.fc2=nn.Linear(11,output_dim)
        self.relu=nn.ReLU(inplace=True)
    def forward(self,x):
        x=self.relu(self.fc1(x))
        x=self.fc2(x)
        return x
net=pca_Net()

In [9]:
import time
import torch.optim as optim
criterion=nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
start = time.time()
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # 获取输入数据
        inputs, label,index = data
        # 清空梯度缓存
        optimizer.zero_grad()
        # print(inputs.shape)
        # print(torch.dist(inputs,torch.tensor(partial_shaps[index]),1))
        outputs = net(inputs).squeeze()
        label=label.squeeze()
        # print(outputs.shape,label.shape)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # 打印统计信息
        running_loss += loss.item()
        if i % 2000 == 1999:
            # 每 2000 次迭代打印一次信息
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i+1, running_loss / 50))
            running_loss = 0.0
print('Finished Training! Total cost time: ', time.time()-start)

Finished Training! Total cost time:  1.8338234424591064


In [10]:
import math
def getDIff(k:int,nums1:np.ndarray,nums2:np.ndarray,alpha=0.2,beta=0.2):
        #排序得到K，K中交集同号数，K中交集异号数，K中非交集数
    nums1=nums1.flatten()
    nums2=nums2.flatten()
    abs_nums1=np.maximum(nums1,-nums1)
    abs_nums2=np.maximum(nums2,-nums2)
    top_k1_index=abs_nums1.argsort()[-k:]
    top_k2_index=abs_nums2.argsort()[-k:]
    inter_index=np.intersect1d(top_k1_index,top_k2_index)
    same_inter=np.sum(nums1[inter_index]*nums2[inter_index]>0)
    score=math.exp((k-(same_inter-alpha*(len(inter_index)-same_inter)-beta*(k-len(inter_index))))*-1)
    return score

In [11]:
torch.save(net.state_dict(),"net.pt")

In [12]:
# # for i,data in enumerate(test_dataloader,0):
# #     x, y, index=data
#     # index+=int(22792*0.7)
#     # print(shaps_mean_pca[index]==y.detach().numpy())
# x,y,index=next(iter(test_dataloader))
# print(index)
# np.where((torch.tensor(shaps_mean_pca).to(torch.float)).detach().numpy()==y.detach().numpy())
# (torch.tensor(shaps_mean_pca).to(torch.float)).detach().numpy()[16371]
# y.detach().numpy()
# 417+int(22792*0.7)

In [13]:

loss1 = 0
loss2 = 0
loss3 = 0
loss4 = 0
loss5 = 0
# needTo=False
for i,data in enumerate(test_dataloader,0):
    # print(x.shape,partial_shaps[index].shape)
    x, y, index=data
    index+=int(22792*0.7)
    # print(torch.dist(x,torch.tensor(partial_shaps[index]),1))

    # print(net.state_dict())
    # print(x1.shape)
    y=y.squeeze()
   
        # if needTo:
        #     if len(index)==1:
        #
        #         x=torch.tensor(partial_shaps[index].transpose(1,2,0))
        #     else:
        #
        #         x=torch.tensor(partial_shaps[index].transpose(0,2,3,1))
    #需要思考
    output_tensor=net.forward(x).squeeze()
    output_tensor=torch.tensor(pca.inverse_transform(output_tensor.detach().numpy()))
    
    if len(index)==1:
        part_mean_shap=torch.tensor(np.mean(all_input_shap[index],axis=0))
    else:
        part_mean_shap=torch.tensor(np.mean(all_input_shap[index],axis=1))
    # print(part_mean_shap.shape)
    # if len(index)==1:
    #     part_mean_shap=torch.tensor(np.mean(all_input_shap[index],axis=0))
    # com_shap=y
    # com_shap=torch.tensor(shaps_mean_pca[index])
    # com_shap=torch.tensor(pca.inverse_transform(com_shap.detach().numpy()))
    # print(all_shap[index].  shape)
    if len(index)==1:
        com_shap=torch.tensor(all_shap[index]).unsqueeze(0)
    else:
        com_shap=torch.tensor(all_shap[index])
    # print(output_tensor.shape,com_shap.shape,part_mean_shap.shape)
    # print(output_tensor.shape,com_shap.shape)
    loss1+=torch.dist(output_tensor,com_shap,2)
    loss2+=torch.dist(part_mean_shap,com_shap,2)
    loss3+=torch.dist(output_tensor,part_mean_shap,2)
    for i in range(len(index)):
        loss4+=getDIff(200,output_tensor[i].detach().numpy(),com_shap[i].detach().numpy())
        loss5+=getDIff(200,part_mean_shap[i].detach().numpy(),com_shap[i].detach().numpy())
print(loss1,loss2,loss3,loss4,loss5)

tensor(481.3336, dtype=torch.float64) tensor(690.7486, dtype=torch.float64) tensor(550.8325, dtype=torch.float64) 5.144519533730801e-95 4.561331580983388e-96
